In [14]:
import pandas as pd
import numpy as np


# Load the uploaded CSV file
file_path = 'Todas_as_predições_Orthology.csv'
predicoes_df = pd.read_csv(file_path)

# Display the first few rows to understand its structure
predicoes_df

,sRNA,Target,E_intaRNA,p_intaRNA,E_Rnaplex,p_Rnaplex,E_TargetRNA3,p_TargetRNA3,Probability_TargetRNA3,Probability_sRNARFTarget
0,srn_0820_sRNA57,SABB_RS00005//SA_RS00145//SAR_RS00005//SAUSA30...,-8.19,0.123395,-40.9,0.649681,-9.36,0.115946,0.103113,0.48841
1,srn_0830_sRNA58,SABB_RS00005//SA_RS00145//SAR_RS00005//SAUSA30...,-8.63,0.114179,-59.5,0.402680,-12.16,0.010360,0.149211,0.57569
2,srn_0700_sRNA45,SABB_RS00005//SA_RS00145//SAR_RS00005//SAUSA30...,-6.44,0.170351,-84.2,0.170348,-7.11,0.022020,0.134363,0.45679
3,srn_0700_sRNA45,SABB_RS00005//SA_RS00145//SAR_RS00005//SAUSA30...,-6.72,0.161519,-55.2,0.455874,-8.30,0.017996,0.145177,0.42785
4,srn_0850_sRNA59,SABB_RS00005//SA_RS00145//SAR_RS00005//SAUSA30...,-7.91,0.129730,-51.1,0.509329,-6.84,0.084467,0.116107,0.43723
...,...,...,...,...,...,...,...,...,...,...
1731055,srn_079,SABB_RS16815//SA_RS15020//SAR_RS15990//SAUSA30...,-8.38,0.119309,-80.1,0.199842,-7.03,0.487171,0.064108,0.46799
1731056,srn_0800_sRNA55,SABB_RS16815//SA_RS15020//SAR_RS15990//SAUSA30...,-17.85,0.027370,-62.0,0.373307,-5.64,0.915588,0.045702,0.49017
1731057,srn_0800_sRNA55,SABB_RS16815//SA_RS15020//SAR_RS15990//SAUSA30...,-17.94,0.027027,-62.5,0.367579,-6.12,0.742282,0.056924,0.49171
1731058,srn_3910_RNAIII,SABB_RS16815//SA_RS15020//SAR_RS15990//SAUSA30...,-2.99,0.354568,-36.1,0.716511,-12.83,0.936856,0.057608,0.54841


In [15]:

unique_srna_count = predicoes_df['sRNA'].dropna().loc[predicoes_df['sRNA'].str.startswith('srn_', na=False)].nunique()

print(f"Number of unique sRNA: {unique_srna_count}")

Number of unique sRNA: 442


In [16]:
predicoes_filtro_outlier_df = predicoes_df[
    (predicoes_df['E_intaRNA'] <= -2.44) &
    (predicoes_df['E_Rnaplex'] <= -32.6) &
    (predicoes_df['E_TargetRNA3'] <= -5.13) &
    (predicoes_df['Probability_TargetRNA3'] >= 0.06) &
    (predicoes_df['Probability_sRNARFTarget'] >= 0.40)
]



In [17]:
predicoes_filtro_outlier_df

,sRNA,Target,E_intaRNA,p_intaRNA,E_Rnaplex,p_Rnaplex,E_TargetRNA3,p_TargetRNA3,Probability_TargetRNA3,Probability_sRNARFTarget
0,srn_0820_sRNA57,SABB_RS00005//SA_RS00145//SAR_RS00005//SAUSA30...,-8.19,0.123395,-40.9,0.649681,-9.36,0.115946,0.103113,0.48841
1,srn_0830_sRNA58,SABB_RS00005//SA_RS00145//SAR_RS00005//SAUSA30...,-8.63,0.114179,-59.5,0.402680,-12.16,0.010360,0.149211,0.57569
2,srn_0700_sRNA45,SABB_RS00005//SA_RS00145//SAR_RS00005//SAUSA30...,-6.44,0.170351,-84.2,0.170348,-7.11,0.022020,0.134363,0.45679
3,srn_0700_sRNA45,SABB_RS00005//SA_RS00145//SAR_RS00005//SAUSA30...,-6.72,0.161519,-55.2,0.455874,-8.30,0.017996,0.145177,0.42785
4,srn_0850_sRNA59,SABB_RS00005//SA_RS00145//SAR_RS00005//SAUSA30...,-7.91,0.129730,-51.1,0.509329,-6.84,0.084467,0.116107,0.43723
...,...,...,...,...,...,...,...,...,...,...
1731036,srn_2020_sRNA171sRNA,SABB_RS16815//SA_RS15020//SAR_RS15990//SAUSA30...,-3.64,0.304428,-68.4,0.303884,-8.41,0.622402,0.065882,0.48275
1731043,srn_0670_Sau6994,SABB_RS16815//SA_RS15020//SAR_RS15990//SAUSA30...,-9.31,0.101507,-76.3,0.230360,-8.01,0.718326,0.065882,0.49896
1731046,srn_0700_sRNA45,SABB_RS16815//SA_RS15020//SAR_RS15990//SAUSA30...,-7.19,0.147931,-77.8,0.217941,-5.96,0.417262,0.080882,0.42171
1731050,srn_0730_sRNA48,SABB_RS16815//SA_RS15020//SAR_RS15990//SAUSA30...,-11.87,0.066602,-78.6,0.211517,-7.08,0.700816,0.064108,0.43092


In [18]:
import pandas as pd

# Define the file paths for the uploaded files
file_paths = [
    'biofilm_vs_plank_N315.deseq2.results.tsv',
    'biofilm_vs_plank_MW2.deseq2.results.tsv',
    'biofilm_vs_plank_MRSA252.deseq2.results.tsv',
    'biofilm_vs_plank_LAC.deseq2.results.tsv',
    'biofilm_vs_plank_BMB.deseq2.results.tsv',
    'biofilm_vs_plank_NRS385.deseq2.results.tsv'
]

DEG_dfs = {}

for file_path in file_paths:
    # Read the file
    df = pd.read_csv(file_path, sep='\t')
    # Filter the dataframe
    filtered_df = df[df['padj'] <= 0.05].copy()
    # Store the filtered dataframe in memory
    DEG_dfs[file_path] = filtered_df





In [19]:
# Initialize an empty dataframe for the outer merge
DEG_all_strains = None

# Extract the base names from file paths and process them for column names
col_names = [
    file_path.replace('biofilm_vs_plank_', '')
    .replace('.deseq2.results.tsv', '')
    for file_path in file_paths
]

# Outer merge the log2FoldChange column from each dataframe
for file_path, col_name in zip(file_paths, col_names):
    df = DEG_dfs[file_path][['gene_id', 'log2FoldChange']].rename(columns={'log2FoldChange': col_name})
    if DEG_all_strains  is None:
        DEG_all_strains  = df
    else:
        DEG_all_strains  = pd.merge(DEG_all_strains , df, on='gene_id', how='outer')

DEG_all_strains

,gene_id,N315,MW2,MRSA252,LAC,BMB,NRS385
0,MW_RS00090,NaN,1.612263,NaN,NaN,NaN,NaN
1,MW_RS00265,NaN,-2.102960,NaN,NaN,NaN,NaN
2,MW_RS00390,NaN,-1.728977,NaN,NaN,NaN,NaN
3,MW_RS00395,NaN,-1.632592,NaN,NaN,NaN,NaN
4,MW_RS00400,NaN,-1.645654,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
3969,srn_9670,NaN,NaN,NaN,NaN,1.838875,NaN
3970,srn_9671,NaN,NaN,NaN,NaN,1.664389,NaN
3971,tRNA,NaN,-1.982558,NaN,NaN,NaN,NaN
3972,tmRNA,NaN,3.798800,NaN,NaN,NaN,NaN


In [20]:

# Replace DEG_all_strains with filtered_merged_df to match the current context
numeric_only_df = DEG_all_strains.select_dtypes(include=[np.number])

# Define a function to check if at least 4 columns in the numeric subset match the signal
def matches_signal_and_status(row):
    positive_count = (row > 0).sum()
    negative_count = (row < 0).sum()
    # Determine if the row should be kept based on signal consistency
    if positive_count >= 4:
        return "upregulated"
    elif negative_count >= 4:
        return "downregulated"
    else:
        return np.nan

# Create a new column for regulation status based on the numeric columns
DEG_all_strains["regulation_status"] = numeric_only_df.apply(matches_signal_and_status, axis=1)

# Filter rows based on the regulation_status column
DEG_all_strains_consistence_df = DEG_all_strains.dropna(subset=["regulation_status"])



In [21]:
nan_status_check = DEG_all_strains_consistence_df['regulation_status'].isna().any()
nan_status_check

False

In [22]:
import pandas as pd

# Add new columns to filtered_Deg_df with default values as NaN
predicoes_filtro_outlier_df['sRNA_DEG'] = None
predicoes_filtro_outlier_df['Target_DEG'] = None

# Create a dictionary mapping gene_id to regulation_status from DEG_all_strains_consistence_df
gene_status_mapping = DEG_all_strains_consistence_df.set_index('gene_id')['regulation_status'].to_dict()

# Populate sRNA_DEG based on sRNA matches
predicoes_filtro_outlier_df['sRNA_DEG'] = predicoes_filtro_outlier_df['sRNA'].map(gene_status_mapping)

# Populate Target_DEG based on target matches
predicoes_filtro_outlier_df['Target_DEG'] = predicoes_filtro_outlier_df['Target'].map(gene_status_mapping)



<ipython-input-22-876a01bd5410>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicoes_filtro_outlier_df['sRNA_DEG'] = None
<ipython-input-22-876a01bd5410>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicoes_filtro_outlier_df['Target_DEG'] = None
<ipython-input-22-876a01bd5410>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [23]:
predicoes_filtro_DEG_df = predicoes_filtro_outlier_df.dropna(subset=['sRNA_DEG', 'Target_DEG'])


In [24]:
predicoes_filtro_DEG_df

,sRNA,Target,E_intaRNA,p_intaRNA,E_Rnaplex,p_Rnaplex,E_TargetRNA3,p_TargetRNA3,Probability_TargetRNA3,Probability_sRNARFTarget,sRNA_DEG,Target_DEG
591,srn_0830_sRNA58,SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...,-7.72,0.134254,-63.8,0.352922,-7.87,0.237547,0.090127,0.55785,upregulated,upregulated
592,srn_0700_sRNA45,SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...,-7.78,0.132805,-69.0,0.297821,-7.50,0.180633,0.092988,0.45779,upregulated,upregulated
593,srn_0700_sRNA45,SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...,-4.57,0.248151,-48.0,0.551160,-11.34,0.197600,0.097091,0.46625,upregulated,upregulated
596,srn_0870_Teg38as,SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...,-6.02,0.184762,-41.4,0.642688,-9.74,0.108549,0.316678,0.46425,upregulated,upregulated
600,srn_0930_Teg76,SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...,-7.24,0.146570,-88.2,0.144861,-7.46,0.166218,0.397414,0.61504,downregulated,upregulated
...,...,...,...,...,...,...,...,...,...,...,...,...
1724558,srn_0710_sRNA46,SABB_RS16760//SA_RS15445//SAR_RS15940//SAUSA30...,-9.38,0.100300,-88.1,0.145460,-7.15,0.003613,0.163614,0.52961,upregulated,upregulated
1724559,srn_0720_sRNA47,SABB_RS16760//SA_RS15445//SAR_RS15940//SAUSA30...,-8.60,0.114780,-64.2,0.348482,-7.74,0.008692,0.160485,0.53802,downregulated,upregulated
1724562,srn_0765_tsr11,SABB_RS16760//SA_RS15445//SAR_RS15940//SAUSA30...,-9.11,0.105051,-39.2,0.673446,-12.74,0.613881,0.072667,0.45306,upregulated,upregulated
1724566,srn_0800_sRNA55,SABB_RS16760//SA_RS15445//SAR_RS15940//SAUSA30...,-10.23,0.086925,-66.4,0.324654,-8.38,0.005636,0.165263,0.50784,upregulated,upregulated


In [26]:
import pandas as pd

# Path to the file
file_path = "Table S12 - nodes.txt"

# Read the file into a DataFrame
Module_df = pd.read_csv(file_path, sep="\t")

# Display the first few rows of the DataFrame
print(Module_df.head())

                                            nodeName  altName   nodeAttr
0                                             RNAseP      NaN  turquoise
1  SABB_RS00005//SA_RS00145//SAR_RS00005//SAUSA30...      NaN        red
2  SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...      NaN    magenta
3  SABB_RS00015//SA_RS00155//SAR_RS00015//SAUSA30...      NaN     salmon
4  SABB_RS00020//SA_RS00160//SAR_RS00020//SAUSA30...      NaN     salmon


In [27]:
# Add new columns to filtered_Deg_df with default values as NaN
predicoes_filtro_DEG_df['sRNA_Module'] = None
predicoes_filtro_DEG_df['Target_Module'] = None

# Create a dictionary mapping gene_id to regulation_status from DEG_all_strains_consistence_df
module_status_mapping = Module_df.set_index('nodeName')['nodeAttr'].to_dict()

# Populate sRNA_DEG based on sRNA matches
predicoes_filtro_DEG_df['sRNA_Module'] = predicoes_filtro_DEG_df['sRNA'].map(module_status_mapping )

# Populate Target_DEG based on target matches
predicoes_filtro_DEG_df['Target_Module'] = predicoes_filtro_DEG_df['Target'].map(module_status_mapping )



<ipython-input-27-e6679b0ebdae>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicoes_filtro_DEG_df['sRNA_Module'] = None
<ipython-input-27-e6679b0ebdae>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicoes_filtro_DEG_df['Target_Module'] = None
<ipython-input-27-e6679b0ebdae>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [28]:
predicoes_filtro_DEG_df

,sRNA,Target,E_intaRNA,p_intaRNA,E_Rnaplex,p_Rnaplex,E_TargetRNA3,p_TargetRNA3,Probability_TargetRNA3,Probability_sRNARFTarget,sRNA_DEG,Target_DEG,sRNA_Module,Target_Module
591,srn_0830_sRNA58,SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...,-7.72,0.134254,-63.8,0.352922,-7.87,0.237547,0.090127,0.55785,upregulated,upregulated,blue,magenta
592,srn_0700_sRNA45,SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...,-7.78,0.132805,-69.0,0.297821,-7.50,0.180633,0.092988,0.45779,upregulated,upregulated,black,magenta
593,srn_0700_sRNA45,SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...,-4.57,0.248151,-48.0,0.551160,-11.34,0.197600,0.097091,0.46625,upregulated,upregulated,black,magenta
596,srn_0870_Teg38as,SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...,-6.02,0.184762,-41.4,0.642688,-9.74,0.108549,0.316678,0.46425,upregulated,upregulated,magenta,magenta
600,srn_0930_Teg76,SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...,-7.24,0.146570,-88.2,0.144861,-7.46,0.166218,0.397414,0.61504,downregulated,upregulated,turquoise,magenta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1724558,srn_0710_sRNA46,SABB_RS16760//SA_RS15445//SAR_RS15940//SAUSA30...,-9.38,0.100300,-88.1,0.145460,-7.15,0.003613,0.163614,0.52961,upregulated,upregulated,darkgreen,cyan
1724559,srn_0720_sRNA47,SABB_RS16760//SA_RS15445//SAR_RS15940//SAUSA30...,-8.60,0.114780,-64.2,0.348482,-7.74,0.008692,0.160485,0.53802,downregulated,upregulated,turquoise,cyan
1724562,srn_0765_tsr11,SABB_RS16760//SA_RS15445//SAR_RS15940//SAUSA30...,-9.11,0.105051,-39.2,0.673446,-12.74,0.613881,0.072667,0.45306,upregulated,upregulated,darkgreen,cyan
1724566,srn_0800_sRNA55,SABB_RS16760//SA_RS15445//SAR_RS15940//SAUSA30...,-10.23,0.086925,-66.4,0.324654,-8.38,0.005636,0.165263,0.50784,upregulated,upregulated,green,cyan


In [29]:
filtered_df = predicoes_filtro_DEG_df[predicoes_filtro_DEG_df['sRNA_Module'] == predicoes_filtro_DEG_df['Target_Module']]


In [30]:
# Number of unique sRNAs
unique_sRNAs = filtered_df['sRNA'].nunique()

# Number of unique genes (targets)
unique_genes = filtered_df['Target'].nunique()

# Mean number of sRNAs predicted for each gene
mean_sRNAs_per_gene = (
    filtered_df.groupby('Target')['sRNA'].nunique().mean()
)

# Output the results
print(f"Number of unique sRNAs: {unique_sRNAs}")
print(f"Number of unique genes: {unique_genes}")
print(f"Mean number of sRNAs per gene: {mean_sRNAs_per_gene:.2f}")

Number of unique sRNAs: 199
Number of unique genes: 1482
Mean number of sRNAs per gene: 9.18


In [31]:
filtered_df

,sRNA,Target,E_intaRNA,p_intaRNA,E_Rnaplex,p_Rnaplex,E_TargetRNA3,p_TargetRNA3,Probability_TargetRNA3,Probability_sRNARFTarget,sRNA_DEG,Target_DEG,sRNA_Module,Target_Module
596,srn_0870_Teg38as,SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...,-6.02,0.184762,-41.4,0.642688,-9.74,0.108549,0.316678,0.46425,upregulated,upregulated,magenta,magenta
689,srn_1640_rsaD,SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...,-6.34,0.173651,-58.5,0.414760,-13.29,0.283788,0.368888,0.56994,downregulated,upregulated,magenta,magenta
842,srn_2800_Sau14,SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...,-8.85,0.109882,-36.0,0.717890,-6.95,0.244141,0.082099,0.56445,downregulated,upregulated,magenta,magenta
843,srn_2790_sRNA224,SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...,-9.47,0.098773,-59.0,0.408697,-6.14,0.446163,0.108160,0.56756,downregulated,upregulated,magenta,magenta
912,srn_0440_rsaK,SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...,-8.76,0.111616,-75.0,0.241519,-9.47,0.208599,0.097694,0.53762,downregulated,upregulated,magenta,magenta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1724322,srn_3510_Teg74,SABB_RS16760//SA_RS15445//SAR_RS15940//SAUSA30...,-7.57,0.137962,-86.9,0.152799,-7.30,0.854847,0.163614,0.52890,upregulated,upregulated,cyan,cyan
1724368,srn_3740_Sau6492,SABB_RS16760//SA_RS15445//SAR_RS15940//SAUSA30...,-9.08,0.105595,-52.7,0.488186,-7.80,0.002935,0.164242,0.52265,downregulated,upregulated,cyan,cyan
1724399,srn_4010_sRNA327,SABB_RS16760//SA_RS15445//SAR_RS15940//SAUSA30...,-7.36,0.143366,-53.2,0.481650,-5.88,0.778866,0.106543,0.56487,downregulated,upregulated,cyan,cyan
1724450,srn_0570_sRNA35,SABB_RS16760//SA_RS15445//SAR_RS15940//SAUSA30...,-7.79,0.132565,-67.0,0.318333,-9.91,0.568498,0.072603,0.55585,downregulated,upregulated,cyan,cyan


In [34]:
import pandas as pd

# Read the edges file into a DataFrame
edges_df = pd.read_csv("Table S13- edges.txt", sep="\t")

# Normalize the 'fromNode' and 'toNode' in edges_df to account for unordered pairs
edges_df['pair'] = edges_df.apply(lambda row: tuple(sorted([row['fromNode'], row['toNode']])), axis=1)

# Map the weight from edges_df to filter_df based on the normalized pair
# First, normalize the pair in filter_df for matching
filtered_df['pair'] = filtered_df.apply(lambda row: tuple(sorted([row['sRNA'], row['Target']])), axis=1)

<ipython-input-34-c71a967117a2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['pair'] = filtered_df.apply(lambda row: tuple(sorted([row['sRNA'], row['Target']])), axis=1)


In [35]:


# Merge weight information
filter_df = filtered_df.merge(edges_df[['pair', 'weight']], on='pair', how='left')

# Sort by Target, then weight (NaN weights go to the bottom of each group)
filter_df = filter_df.sort_values(by=['Target', 'weight'], ascending=[True, False])

# Remove NaN rows and keep the first (highest-weight) entry for each Target
filter_df = (
    filter_df[~filter_df['weight'].isna()]  # Exclude rows with NaN weights
    .drop_duplicates(subset='Target', keep='first')  # Keep the highest-weight row per Target
    .reset_index(drop=True)
)

# Optional: Drop the 'pair' column if no longer needed
filter_df = filter_df.drop(columns=['pair'])


In [36]:
filter_df

,sRNA,Target,E_intaRNA,p_intaRNA,E_Rnaplex,p_Rnaplex,E_TargetRNA3,p_TargetRNA3,Probability_TargetRNA3,Probability_sRNARFTarget,sRNA_DEG,Target_DEG,sRNA_Module,Target_Module,weight
0,srn_1640_rsaD,SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...,-6.34,0.173651,-58.5,0.414760,-13.29,0.283788,0.368888,0.56994,downregulated,upregulated,magenta,magenta,0.318457
1,srn_1080_RsaX05,SABB_RS00015//SA_RS00155//SAR_RS00015//SAUSA30...,-5.13,0.220775,-67.2,0.316243,-6.77,0.212047,0.090047,0.56670,downregulated,upregulated,salmon,salmon,0.420139
2,srn_1250_sRNA110,SABB_RS00020//SA_RS00160//SAR_RS00020//SAUSA30...,-7.16,0.148755,-49.8,0.526743,-6.64,0.441926,0.297964,0.57910,upregulated,upregulated,salmon,salmon,0.384638
3,srn_3780_Teg13,SABB_RS00025//SA_RS00165//SAR_RS00025//SAUSA30...,-9.64,0.095965,-68.2,0.305922,-5.46,0.595968,0.072707,0.50996,upregulated,upregulated,yellow,yellow,0.427581
4,srn_2910_Teg63,SABB_RS00030//SA_RS00170//SAR_RS00030//SAUSA30...,-5.52,0.203983,-47.3,0.560739,-9.83,0.120370,0.098568,0.50379,upregulated,downregulated,pink,pink,0.251217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463,srn_3350_sRNA266,SABB_RS16285//SA_RS15060//SAR_RS15585//SAUSA30...,-7.03,0.152393,-72.4,0.264947,-9.71,0.716646,0.079854,0.52047,upregulated,downregulated,tan,tan,0.230532
1464,srn_2300_sRNA193,SABB_RS16420//SA_RS15155//SAR_RS15670//SAUSA30...,-3.90,0.287100,-78.1,0.215516,-10.45,0.000004,0.247066,0.51587,upregulated,upregulated,blue,blue,0.479406
1465,srn_3580_SprA1,SABB_RS16470//SAR_RS03580//SAUSA300_RS15545//M...,-10.86,0.078359,-79.9,0.201371,-9.73,0.805983,0.083272,0.43048,upregulated,upregulated,blue,blue,0.406915
1466,srn_1540_Teg48,SABB_RS16565//SA_RS15245//SAR_RS15735//SAUSA30...,-3.94,0.284554,-34.2,0.742557,-15.03,0.787183,0.066150,0.55778,upregulated,upregulated,brown,brown,0.221896


In [37]:
filter_df.to_csv('filtered_weight.csv', index=False)

In [40]:
filter_df

,sRNA,Target,E_intaRNA,p_intaRNA,E_Rnaplex,p_Rnaplex,E_TargetRNA3,p_TargetRNA3,Probability_TargetRNA3,Probability_sRNARFTarget,sRNA_DEG,Target_DEG,sRNA_Module,Target_Module,weight
0,srn_1640_rsaD,SABB_RS00010//SA_RS00150//SAR_RS00010//SAUSA30...,-6.34,0.173651,-58.5,0.414760,-13.29,0.283788,0.368888,0.56994,downregulated,upregulated,magenta,magenta,0.318457
1,srn_1080_RsaX05,SABB_RS00015//SA_RS00155//SAR_RS00015//SAUSA30...,-5.13,0.220775,-67.2,0.316243,-6.77,0.212047,0.090047,0.56670,downregulated,upregulated,salmon,salmon,0.420139
2,srn_1250_sRNA110,SABB_RS00020//SA_RS00160//SAR_RS00020//SAUSA30...,-7.16,0.148755,-49.8,0.526743,-6.64,0.441926,0.297964,0.57910,upregulated,upregulated,salmon,salmon,0.384638
3,srn_3780_Teg13,SABB_RS00025//SA_RS00165//SAR_RS00025//SAUSA30...,-9.64,0.095965,-68.2,0.305922,-5.46,0.595968,0.072707,0.50996,upregulated,upregulated,yellow,yellow,0.427581
4,srn_2910_Teg63,SABB_RS00030//SA_RS00170//SAR_RS00030//SAUSA30...,-5.52,0.203983,-47.3,0.560739,-9.83,0.120370,0.098568,0.50379,upregulated,downregulated,pink,pink,0.251217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463,srn_3350_sRNA266,SABB_RS16285//SA_RS15060//SAR_RS15585//SAUSA30...,-7.03,0.152393,-72.4,0.264947,-9.71,0.716646,0.079854,0.52047,upregulated,downregulated,tan,tan,0.230532
1464,srn_2300_sRNA193,SABB_RS16420//SA_RS15155//SAR_RS15670//SAUSA30...,-3.90,0.287100,-78.1,0.215516,-10.45,0.000004,0.247066,0.51587,upregulated,upregulated,blue,blue,0.479406
1465,srn_3580_SprA1,SABB_RS16470//SAR_RS03580//SAUSA300_RS15545//M...,-10.86,0.078359,-79.9,0.201371,-9.73,0.805983,0.083272,0.43048,upregulated,upregulated,blue,blue,0.406915
1466,srn_1540_Teg48,SABB_RS16565//SA_RS15245//SAR_RS15735//SAUSA30...,-3.94,0.284554,-34.2,0.742557,-15.03,0.787183,0.066150,0.55778,upregulated,upregulated,brown,brown,0.221896
